# Return 16s rRNA region from Genomes
In this step, I want to simulate amplification of 16s rRNA genes from genome isolates using the primer pairs used for NGS.

This notebook uses the bgc_analytics environment, see: [../env/bgc_analytics.yml](../env/bgc_analytics.yml)

* **Input:** 
    * Path to genomes (.gbk) folder = "../data/genomes"
* **Output:**
    * [../data/genomes/16sgenome.fasta](../data/genomes/16sgenome.fasta)

In [1]:

# load library
import os
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
import sys

In [2]:
# find 16s based on annotation
def find_16s(path):
    records = []
    ctr = 0
    records_list = SeqIO.parse(open(path, 'r'), "genbank")
    for rec in records_list:
        for feat in rec.features :
            if feat.type == "rRNA":
                if feat.qualifiers['product'][0] == '16S ribosomal RNA':
                    print('contig :', rec.id)
                    start = feat.location.start - 1
                    end = feat.location.end - 1
                    strand = feat.location.strand
                    #print('product :', feat.qualifiers['product'][0])
                    print(start, ':', end, '| strand :', strand, '| length :', len(rec.seq[start:end]))
                    record = SeqRecord(rec.seq[start:end],
                                       id=rec.id, 
                                       name="16S ribosomal RNA",
                                       description="strand:"+str(strand)+"_loc:"+str(start)+"-"+str(end))
                    records.append((record))
    return records

# read genomes, append 16s record to *result*
path = '../data/genomes/'
x = [os.path.join(path, i) for i in os.listdir('../data/genomes/') if i.endswith('.gbk')]
result = []
for i in x:
    result.append(find_16s(i))

In [3]:
# find v3-v4 region using part of the primer (p1 and p2)
def v3(seq):
    p1 = Seq("CCTACGGG") #CCTACGGGNGGCWGCAG
    p2 = Seq("GACTAC").reverse_complement() #"GACTACHVGGGTATCTAATCC"
    try:
        start = seq.index(p1)
        end = seq.index(p2)
    except:
        end = seq.index(p1.reverse_complement())
        start = seq.index(p2.reverse_complement())
    sequence = seq[start:end] 
    return sequence

# full primers
p1_full = 'CCTACGGGNGGCWGCAG'
p2_full = "GACTACHVGGGTATCTAATCC"

# return clean ASV from the 16s (minus the primers)
for record in result:
    for seq in record:
        strand = seq.description.split('_')
        if strand[0] == 'strand:-1':
            v3v4 =  v3(seq.seq.reverse_complement())[len(p1_full):-len(p2_full)]
            s = str('>'+seq.id+'-'+seq.description+'\n'+v3v4)
        else:
            v3v4 = v3(seq.seq)[len(p1_full):-len(p2_full)]
            s = str('>'+seq.id+'-'+seq.description+'\n'+v3v4)
        print(len(v3v4), s)

In [6]:
# write result to a fasta files
original_stdout = sys.stdout # Save a reference to the original standard output

p1_full = 'CCTACGGGNGGCWGCAG'
p2_full = "GACTACHVGGGTATCTAATCC"

with open('../data/genomes/16sgenome.fasta', 'w') as f: # output fasta files
    sys.stdout = f # Change the standard output to the file we created.
    for record in result:
        for seq in record:
            strand = seq.description.split('_')
            prev = v3v4 # counter for previous result
            if strand[0] == 'strand:-1':
                v3v4 =  v3(seq.seq.reverse_complement())[len(p1_full):-len(p2_full)]
                s = str('>'+seq.id+'-'+seq.description+'\n'+v3v4)
                #s = str('>'+seq.id+'\n'+v3v4)
            else:
                v3v4 = v3(seq.seq)[len(p1_full):-len(p2_full)]
                s = str('>'+seq.id+'-'+seq.description+'\n'+v3v4)
                #s = str('>'+seq.id+'\n'+v3v4)
            # there are 6 copies in genome, if result are the same then skip
            if v3v4 == prev:
                pass
            else:
                print(s)
    sys.stdout = original_stdout # Reset the standard output to its original value

In [ ]:
# join the resulting fasta with ASVs from QIIME2
#! mkdir ../data/combined_tree
#! cat ../data/genomes/16sgenome.fasta ../data/qiime2/filtered/dna-sequences.fasta > ../data/combined_tree/combined.fasta